In [28]:
import pandas as pd

df = pd.read_csv('Housing.csv')
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [29]:
df.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [30]:
df = df.drop(['hotwaterheating','airconditioning', 'furnishingstatus'], axis = 1)

In [31]:
df = pd.get_dummies(df, columns = ["mainroad", "guestroom", "basement", "prefarea"])
df

,price,area,bedrooms,bathrooms,stories,parking,mainroad_no,mainroad_yes,guestroom_no,guestroom_yes,basement_no,basement_yes,prefarea_no,prefarea_yes
0,13300000,7420,4,2,3,2,0,1,1,0,1,0,0,1
1,12250000,8960,4,4,4,3,0,1,1,0,1,0,1,0
2,12250000,9960,3,2,2,2,0,1,1,0,0,1,0,1
3,12215000,7500,4,2,2,3,0,1,1,0,0,1,0,1
4,11410000,7420,4,1,2,2,0,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,2,0,1,1,0,0,1,1,0
541,1767150,2400,3,1,1,0,1,0,1,0,1,0,1,0
542,1750000,3620,2,1,1,0,0,1,1,0,1,0,1,0
543,1750000,2910,3,1,1,0,1,0,1,0,1,0,1,0


In [32]:
X = df.drop('price', axis = 1).to_numpy()
y = df['price'].to_numpy()

In [33]:
import sklearn
print(sklearn.__version__)


1.2.1


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Define the model
gbr = GradientBoostingRegressor(random_state=42)

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [500, 1000],              # Number of trees in the ensemble
    'learning_rate': [0.1, 0.2],              # Learning rate
    'max_depth': [6, 9],                       # Maximum depth of trees
    'min_samples_leaf': [16, 32],              # Minimum samples per leaf
    'max_features': [0.1, 0.2],                # Fraction of features to consider
    'loss': ['huber', 'ls'],                   # Loss function to use ('huber' or 'least squares')
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, 
                           cv=3, scoring='neg_mean_squared_error', 
                           verbose=2, n_jobs=-1)

# Fit grid search to find the best model
grid_search.fit(X_train, y_train)

# Get the best parameters from grid search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Use the best model found by GridSearchCV
best_gbr = grid_search.best_estimator_

# Predicting on the test set
y_pred = best_gbr.predict(X_test)

# Calculate Mean Squared Error
gbr_MSE = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", gbr_MSE)

# Calculate Mean Absolute Error
gbr_MAE = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", gbr_MAE)

# Calculate R-squared score
gbr_r2 = r2_score(y_test, y_pred)
print("R-squared score:", gbr_r2)


Fitting 3 folds for each of 64 candidates, totalling 192 fits


c:\Users\yenye\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
96 fits failed out of a total of 192.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
78 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\yenye\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\yenye\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 420, in fit
    self._validate_params()
  File "c:\Users\yenye\anaconda3\lib\site-packages\sklearn\base.py", line 581, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\yenye\anaconda3\lib\site-packages\s

Best Parameters: {'learning_rate': 0.1, 'loss': 'huber', 'max_depth': 6, 'max_features': 0.1, 'min_samples_leaf': 32, 'n_estimators': 500}
Mean Squared Error: 1655619347309.577
Mean Absolute Error: 960877.6006180634
R-squared score: 0.6155436731571977


In [37]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(X_train, y_train)

LR_y_pred = LR_model.predict(X_test)

LR_MSE = mean_squared_error(y_test, LR_y_pred)
print(LR_MSE)

LR_MAE = mean_absolute_error(y_test, LR_y_pred)
print("Mean Absolute Error:", LR_MAE)


LR_r2 = r2_score(y_test, LR_y_pred)
print("R-squared score:", LR_r2)

1855270502842.4475
Mean Absolute Error: 1006370.4223193768
R-squared score: 0.5691820804210288


In [38]:
from sklearn.tree import DecisionTreeRegressor

dt_reg =DecisionTreeRegressor(random_state= 42)

dt_reg.fit(X_train, y_train)

dt_y_pred = dt_reg.predict(X_test)
dt_MSE = mean_squared_error(y_test, dt_y_pred)
print(dt_MSE)

dt_MAE = mean_absolute_error(y_test, dt_y_pred)
print("Mean Absolute Error:", dt_MAE)

# Calculate R-squared score
dt_r2 = r2_score(y_test, dt_y_pred)
print("R-squared score:", dt_r2)

3346410099938.8887
Mean Absolute Error: 1295092.7642276424
R-squared score: 0.22292008895472637


In [39]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor(n_estimators=100)

RF_model.fit(X_train, y_train)

RF_y_pred = RF_model.predict(X_test)
RF_MSE = mean_squared_error(y_test, RF_y_pred)
print(RF_MSE)

# Calculate Mean Absolute Error
RF_MAE = mean_absolute_error(y_test, RF_y_pred)
print("Mean Absolute Error:", gbr_MAE)

# Calculate R-squared score
RF_r2 = r2_score(y_test, RF_y_pred)
print("R-squared score:", RF_r2)

1927499501153.545
Mean Absolute Error: 960877.6006180634
R-squared score: 0.5524095684137582


In [40]:
from sklearn.neighbors import KNeighborsRegressor
KN_model = KNeighborsRegressor(n_neighbors=5)

KN_model.fit(X_train, y_train)

KN_y_pred = KN_model.predict(X_test)
KN_MSE = mean_squared_error(y_test, KN_y_pred)
print(KN_MSE)

KN_MAE = mean_absolute_error(y_test, KN_y_pred)
print("Mean Absolute Error:", gbr_MAE)

KN_r2 = r2_score(y_test, KN_y_pred)
print("R-squared score:", KN_r2)

2983137843038.073
Mean Absolute Error: 960877.6006180634
R-squared score: 0.3072766276475951


In [41]:
import joblib
# Save the trained model using joblib
joblib.dump(best_gbr, 'Gradient_Boosting_Regressor.pkl')

['Gradient_Boosting_Regressor.pkl']